In [56]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report, roc_curve, auc, confusion_matrix
from sklearn.preprocessing import label_binarize
from transformers import BertTokenizer, BertForMaskedLM, pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
from nltk.corpus import stopwords
import re
from imblearn.over_sampling import SMOTE
import random
import torch
from transformers import MarianMTModel, MarianTokenizer
from concurrent.futures import ThreadPoolExecutor

In [58]:
data = pd.read_csv('./all_data.csv')
data.drop_duplicates(subset='uu_usl_name', keep='first', inplace=True)
data.drop('SERVICE_CLASS_CONFIRMED', axis=1, inplace=True)
data.tail()

,uu_usl_name,CLASS_ID,CNT_DIST_USL_BY_CLASS,CNT_NUM_USL
81601,Прием врачом-онкологом первичный амбулаторный,9000,169,1
81602,Проводниковая анестезия,9000,169,1
81603,Проводниковая анестезия (блокада по Оберсту-Лу...,9000,169,1
81604,Проводниковая анестезия Sol. Ultracani D.S,9000,169,1
81605,Удаление новообразования мягких тканей под мес...,9000,169,1


In [51]:


# Загрузка моделей для перевода
model_name_ru_to_en = 'Helsinki-NLP/opus-mt-ru-en'
model_name_en_to_ru = 'Helsinki-NLP/opus-mt-en-ru'

tokenizer_ru_to_en = MarianTokenizer.from_pretrained(model_name_ru_to_en)
model_ru_to_en = MarianMTModel.from_pretrained(model_name_ru_to_en)

tokenizer_en_to_ru = MarianTokenizer.from_pretrained(model_name_en_to_ru)
model_en_to_ru = MarianMTModel.from_pretrained(model_name_en_to_ru)

# Функция для перевода текста
def translate(text, model, tokenizer, target_language="en"):
    # Токенизируем текст
    tokenized_text = tokenizer.prepare_seq2seq_batch([text], return_tensors='pt')
    # Переводим текст
    translated = model.generate(**tokenized_text)
    # Декодируем переведенный текст
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    return translated_text

# Функция для back-translation
def back_translate(text, source_language="ru", target_language="en"):
    # Переводим с русского на английский
    translated_to_en = translate(text, model_ru_to_en, tokenizer_ru_to_en, target_language)
    # Переводим с английского обратно на русский
    back_translated_to_ru = translate(translated_to_en, model_en_to_ru, tokenizer_en_to_ru, source_language)
    return back_translated_to_ru

# Кэширование переводов
translation_cache = {}

# Функция для аугментации с использованием back-translation
def augment_text(text):
    # Если текст уже переведен, используем кэшированный перевод
    if text in translation_cache:
        return translation_cache[text]
    
    # Иначе, выполняем back-translation
    back_translated_text = back_translate(text)
    translation_cache[text] = back_translated_text
    return back_translated_text

# Выборочная аугментация и многопоточность
def selective_augmentation(texts, augmentation_probability=0.1, num_threads=10):
    augmented_texts = []
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = []
        for text in texts:
            if random.random() < augmentation_probability:
                futures.append(executor.submit(augment_text, text))
            else:
                augmented_texts.append(text)
        
        # Дождаться завершения всех операций перевода
        for future in futures:
            augmented_texts.append(future.result())
    
    return augmented_texts

# Пример использования
# texts_to_augment = ["Пример текста для аугментации", "Еще один пример текста"]
# augmented_texts = selective_augmentation(texts_to_augment)


In [52]:
class_counts = data['CLASS_ID'].value_counts()
minor_classes = class_counts[class_counts < 10].index

In [53]:
# Создайте новый DataFrame с аугментированными данными
augmented_data = []
for class_id in minor_classes:
    class_data = data[data['CLASS_ID'] == class_id]

    # Собираем тексты для аугментации
    texts_to_augment = class_data['uu_usl_name'].tolist()

    # Применяем selective_augmentation
    augmented_texts = selective_augmentation(texts_to_augment)

    # Добавляем аугментированные тексты в augmented_data
    for augmented_text in augmented_texts:
        augmented_data.append({'CLASS_ID': class_id, 'uu_usl_name': augmented_text})

# Создайте новый DataFrame с аугментированными данными
new_data = pd.DataFrame(augmented_data)

# Объедините оригинальный DataFrame и новый DataFrame с аугментированными данными
final_data = pd.concat([data, new_data])

/Users/borisyakunin1311icloud.com/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3974: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


KeyboardInterrupt: 

In [54]:
class_counts = final_data['CLASS_ID'].value_counts()
minor_classes_count = class_counts.min()
minor_classes_count

7

In [55]:
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"[^а-яА-Яa-zA-Z0-9]", " ", text)  # Удаляем все символы, кроме букв и цифр
    words = text.split()
    words = [w for w in words if w not in stop_words]  # Удаляем стоп-слова
    text = " ".join(words)
    return text
# Предполагается, что текстовый столбец называется 'text'
final_data['uu_usl_name'] = final_data['uu_usl_name'].apply(preprocess_text)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/borisyakunin1311icloud.com/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [57]:
X = final_data['uu_usl_name']
y = final_data["CLASS_ID"]

X_train, X_test, y_train, y_test = train_test_split(final_data['uu_usl_name'], final_data["CLASS_ID"], test_size=0.2, random_state=42)

# Препроцессинг
X_train_preprocessed = [preprocess_text(text) for text in X_train]
X_test_preprocessed = [preprocess_text(text) for text in X_test]

# Векторизация
vectorizer = TfidfVectorizer(max_features=1000)
X_train_vectors = vectorizer.fit_transform(X_train_preprocessed)
X_test_vectors = vectorizer.transform(X_test_preprocessed)

# Создаем объект SMOTE
smote = SMOTE(sampling_strategy='auto', k_neighbors=min(minor_classes_count - 1, 5), random_state=42)

# Применяем SMOTE для балансировки классов
X_resampled, y_resampled = smote.fit_resample(X_train_vectors, y_train)





ValueError: Expected n_neighbors <= n_samples,  but n_samples = 3, n_neighbors = 6

In [59]:
# Обучение модели
classifier = RandomForestClassifier(random_state=42)
classifier.fit(X_resampled_vectors, y_resampled)
y_pred = classifier.predict(vectorizer.transform(X_test))

print(classification_report(y_test, y_pred))

# # Обучение модели
# classifier = RandomForestClassifier(random_state=42, oob_score=True)
# # Обучение модели
# classifier.fit(X_train, y_train)

# # Оценка качества модели с использованием out-of-bag score
# oob_score = classifier.oob_score_


NameError: name 'X_resampled_vectors' is not defined

In [ ]:
# Предсказание и оценка
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10,10))
sns.heatmap(cm, annot=True, fmt="d")
plt.show()
# y_pred = classifier.predict(vectorizer.transform(X_test))
# print(f"Out-of-Bag Score (оценка модели): {oob_score}")